In [2]:
# Import necessary libraries
import pandas as pd
from faker import Faker

# Initialize the Faker object
fake = Faker()

# Function to generate fake sneaker data
def generate_sneaker_data(num_rows):
    data = {
        'id': [fake.uuid4() for _ in range(num_rows)],
        'brand': [fake.company() for _ in range(num_rows)],
        'model': [fake.word() for _ in range(num_rows)],
        'size': [fake.random_int(min=6, max=14) for _ in range(num_rows)],
        'price': [round(fake.random_number(digits=4, fix_len=True) / 100, 2) for _ in range(num_rows)],
        'release_date': [fake.date_this_decade() for _ in range(num_rows)],
        'color': [fake.color_name() for _ in range(num_rows)]
    }
    return pd.DataFrame(data)

# Generate 100 rows of fake sneaker data
sneaker_df = generate_sneaker_data(100)

# Display the first few rows of the dataframe
sneaker_df.head()


,id,brand,model,size,price,release_date,color
0,877162ad-9e59-4914-b3ba-e3be8ca2b932,Hudson Group,ask,13,83.55,2023-07-21,OrangeRed
1,cc87fa77-b7ef-4401-bc16-26cb65fd905e,Johnson PLC,cause,10,25.01,2023-09-09,DarkTurquoise
2,9b1234c1-5a67-4006-ad6b-e6c2fa20005d,Henderson-Patton,subject,11,55.07,2024-07-23,Silver
3,54e0bcb0-fbbe-470d-9bde-97839e71b059,Clark Inc,character,8,26.12,2021-09-04,MediumAquaMarine
4,21ac1913-2159-4e66-adef-6759cc5d784d,Bennett-Lopez,including,14,86.02,2021-06-29,DarkCyan


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize PySpark session
spark = SparkSession.builder.appName("ChancellorDataEngineering").getOrCreate()

# Convert pandas DataFrame to PySpark DataFrame
sneaker_spark_df = spark.createDataFrame(sneaker_df)

# Display PySpark DataFrame schema
sneaker_spark_df.printSchema()

# Show first few rows of the DataFrame
sneaker_spark_df.show(5)


root
 |-- id: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- size: long (nullable = true)
 |-- price: double (nullable = true)
 |-- release_date: date (nullable = true)
 |-- color: string (nullable = true)

+--------------------+----------------+---------+----+-----+------------+----------------+
|                  id|           brand|    model|size|price|release_date|           color|
+--------------------+----------------+---------+----+-----+------------+----------------+
|877162ad-9e59-491...|    Hudson Group|      ask|  13|83.55|  2023-07-21|       OrangeRed|
|cc87fa77-b7ef-440...|     Johnson PLC|    cause|  10|25.01|  2023-09-09|   DarkTurquoise|
|9b1234c1-5a67-400...|Henderson-Patton|  subject|  11|55.07|  2024-07-23|          Silver|
|54e0bcb0-fbbe-470...|       Clark Inc|character|   8|26.12|  2021-09-04|MediumAquaMarine|
|21ac1913-2159-4e6...|   Bennett-Lopez|including|  14|86.02|  2021-06-29|        DarkCyan|
+-------

In [4]:
# Remove rows with null values
cleaned_df = sneaker_spark_df.dropna()

# Rename columns for consistency
transformed_df = cleaned_df.withColumnRenamed('release_date', 'releaseDate') \
                           .withColumnRenamed('id', 'sneakerID')

# Show cleaned and transformed data
transformed_df.show(5)


+--------------------+----------------+---------+----+-----+-----------+----------------+
|           sneakerID|           brand|    model|size|price|releaseDate|           color|
+--------------------+----------------+---------+----+-----+-----------+----------------+
|877162ad-9e59-491...|    Hudson Group|      ask|  13|83.55| 2023-07-21|       OrangeRed|
|cc87fa77-b7ef-440...|     Johnson PLC|    cause|  10|25.01| 2023-09-09|   DarkTurquoise|
|9b1234c1-5a67-400...|Henderson-Patton|  subject|  11|55.07| 2024-07-23|          Silver|
|54e0bcb0-fbbe-470...|       Clark Inc|character|   8|26.12| 2021-09-04|MediumAquaMarine|
|21ac1913-2159-4e6...|   Bennett-Lopez|including|  14|86.02| 2021-06-29|        DarkCyan|
+--------------------+----------------+---------+----+-----+-----------+----------------+
only showing top 5 rows



In [6]:
import mysql.connector

# Establish connection to MySQL database
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',  # Your MySQL username
    password='Ch@ncetek869219',  # Your MySQL password
    database='sneaker_db',  # Your database name
)

# Create a cursor object to execute SQL queries
cursor = db_connection.cursor()

# Define the SQL query to create a table
create_table_query = '''
CREATE TABLE IF NOT EXISTS sneakers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    brand VARCHAR(50),
    model VARCHAR(50),
    size INT,
    price DECIMAL(10, 2),
    stock INT
);
'''

# Execute the SQL query to create the table
cursor.execute(create_table_query)

# Commit the transaction
db_connection.commit()

# Close the cursor and connection
cursor.close()
db_connection.close()

print("Table created successfully!")


Table created successfully!


In [7]:
# Insert transformed data into MySQL
for row in transformed_df.collect():
    insert_query = '''
    INSERT INTO sneakers (sneakerID, brand, model, size, price, releaseDate, color) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, (row.sneakerID, row.brand, row.model, row.size, row.price, row.releaseDate, row.color))
    db_connection.commit()

# Close connection
cursor.close()
db_connection.close()


NameError: name 'transformed_df' is not defined

In [8]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("SneakerDataTransformation") \
    .getOrCreate()


In [9]:
# Sample raw data
raw_data = [
    (1, 'Nike', 'Air Max', 42, 120.0, '2024-01-01', 'Red'),
    (2, 'Adidas', 'Ultraboost', 43, 150.0, '2023-12-15', 'Black'),
    (3, 'Puma', 'RS-X', 40, 90.0, '2023-11-20', 'White'),
]

# Create a DataFrame
columns = ['sneakerID', 'brand', 'model', 'size', 'price', 'releaseDate', 'color']
raw_df = spark.createDataFrame(raw_data, columns)

# Perform transformations
transformed_df = raw_df.withColumnRenamed("releaseDate", "release_date")

# Show the transformed DataFrame (optional)
transformed_df.show()


+---------+------+----------+----+-----+------------+-----+
|sneakerID| brand|     model|size|price|release_date|color|
+---------+------+----------+----+-----+------------+-----+
|        1|  Nike|   Air Max|  42|120.0|  2024-01-01|  Red|
|        2|Adidas|Ultraboost|  43|150.0|  2023-12-15|Black|
|        3|  Puma|      RS-X|  40| 90.0|  2023-11-20|White|
+---------+------+----------+----+-----+------------+-----+



In [10]:
import mysql.connector

# Establish connection to MySQL database
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',  # Your MySQL username
    password='Ch@ncetek869219',  # Your MySQL password
    database='sneaker_db',  # Your database name
)

cursor = db_connection.cursor()

# Insert transformed data into MySQL
for row in transformed_df.collect():
    insert_query = '''
    INSERT INTO sneakers (sneakerID, brand, model, size, price, release_date, color) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, (row.sneakerID, row.brand, row.model, row.size, row.price, row.release_date, row.color))

# Commit the transaction
db_connection.commit()

# Close the cursor and connection
cursor.close()
db_connection.close()

print("Data inserted successfully!")


ProgrammingError: 1054 (42S22): Unknown column 'sneakerID' in 'field list'

In [11]:
DESCRIBE sneakers;



SyntaxError: invalid syntax (1365690095.py, line 1)

In [12]:
DESCRIBE sneakers;


SyntaxError: invalid syntax (917396482.py, line 1)

In [13]:
import mysql.connector

# Establish connection to MySQL database
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',  # Your MySQL username
    password='Ch@ncetek869219',  # Your MySQL password
    database='sneaker_db',  # Your database name
)

cursor = db_connection.cursor()


In [14]:
# Execute the DESCRIBE query
cursor.execute("DESCRIBE sneakers;")

# Fetch all results
table_description = cursor.fetchall()

# Print the table description
for column in table_description:
    print(column)


('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('brand', 'varchar(50)', 'YES', '', None, '')
('model', 'varchar(50)', 'YES', '', None, '')
('size', 'int', 'YES', '', None, '')
('price', 'decimal(10,2)', 'YES', '', None, '')
('stock', 'int', 'YES', '', None, '')


In [15]:
# Insert transformed data into MySQL
for row in transformed_df.collect():
    insert_query = '''
    INSERT INTO sneakers (id, brand, model, size, price, release_date, color) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, (row.id, row.brand, row.model, row.size, row.price, row.release_date, row.color))


AttributeError: id

In [16]:
# Print the schema of the transformed DataFrame to verify the column names
transformed_df.printSchema()



root
 |-- sneakerID: long (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- size: long (nullable = true)
 |-- price: double (nullable = true)
 |-- release_date: string (nullable = true)
 |-- color: string (nullable = true)



In [17]:
# Insert transformed data into MySQL
for row in transformed_df.collect():
    insert_query = '''
    INSERT INTO sneakers (sneakerID, brand, model, size, price, release_date, color) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''
    cursor.execute(insert_query, (row.sneakerID, row.brand, row.model, row.size, row.price, row.release_date, row.color))

# Commit the transaction
db_connection.commit()

# Close the cursor and connection
cursor.close()
db_connection.close()

print("Data inserted successfully!")


ProgrammingError: 1054 (42S22): Unknown column 'sneakerID' in 'field list'

In [18]:
DESCRIBE sneakers;


SyntaxError: invalid syntax (917396482.py, line 1)

In [19]:
cursor.execute("DESCRIBE sneakers;")

In [20]:
ALTER TABLE sneakers ADD COLUMN sneakerID INT PRIMARY KEY AUTO_INCREMENT;


SyntaxError: invalid syntax (4242669380.py, line 1)

In [21]:
import mysql.connector

# Establish connection to MySQL database
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',  # Replace with your MySQL username
    password='Ch@ncetek869219',  # Replace with your MySQL password
    database='sneaker_db'  # Replace with your database name
)

# Create a cursor object to execute queries
cursor = db_connection.cursor()

# Execute the ALTER TABLE query to add the sneakerID column
alter_table_query = '''
ALTER TABLE sneakers 
ADD COLUMN sneakerID INT PRIMARY KEY AUTO_INCREMENT;
'''
cursor.execute(alter_table_query)

# Commit the transaction
db_connection.commit()

# Verify that the sneakerID column was added
cursor.execute("DESCRIBE sneakers;")
for row in cursor.fetchall():
    print(row)

# Close the cursor and connection
cursor.close()
db_connection.close()


ProgrammingError: 1075 (42000): Incorrect table definition; there can be only one auto column and it must be defined as a key

In [22]:
cursor.execute("DESCRIBE sneakers;")
for row in cursor.fetchall():
    print(row)


('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('brand', 'varchar(50)', 'YES', '', None, '')
('model', 'varchar(50)', 'YES', '', None, '')
('size', 'int', 'YES', '', None, '')
('price', 'decimal(10,2)', 'YES', '', None, '')
('stock', 'int', 'YES', '', None, '')


In [23]:
# Drop the existing AUTO_INCREMENT column (if any)
drop_auto_increment_query = '''
ALTER TABLE sneakers 
MODIFY COLUMN existing_column_name INT;
'''
cursor.execute(drop_auto_increment_query)
db_connection.commit()


ProgrammingError: 1054 (42S22): Unknown column 'existing_column_name' in 'sneakers'

In [24]:
cursor.execute("DESCRIBE sneakers;")
for row in cursor.fetchall():
    print(row)


('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('brand', 'varchar(50)', 'YES', '', None, '')
('model', 'varchar(50)', 'YES', '', None, '')
('size', 'int', 'YES', '', None, '')
('price', 'decimal(10,2)', 'YES', '', None, '')
('stock', 'int', 'YES', '', None, '')


In [25]:
# Replace 'actual_column_name' with the correct column name
drop_auto_increment_query = '''
ALTER TABLE sneakers 
MODIFY COLUMN actual_column_name INT;
'''
cursor.execute(drop_auto_increment_query)
db_connection.commit()


ProgrammingError: 1054 (42S22): Unknown column 'actual_column_name' in 'sneakers'

In [26]:
# Check the structure of the 'sneakers' table
cursor.execute("DESCRIBE sneakers;")
columns = cursor.fetchall()

# Print the columns to see their names and types
for column in columns:
    print(column)


('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('brand', 'varchar(50)', 'YES', '', None, '')
('model', 'varchar(50)', 'YES', '', None, '')
('size', 'int', 'YES', '', None, '')
('price', 'decimal(10,2)', 'YES', '', None, '')
('stock', 'int', 'YES', '', None, '')


In [27]:
# Replace 'your_column_name' with the correct column name identified in the previous step
drop_auto_increment_query = '''
ALTER TABLE sneakers 
MODIFY COLUMN your_column_name INT;
'''
cursor.execute(drop_auto_increment_query)
db_connection.commit()


ProgrammingError: 1054 (42S22): Unknown column 'your_column_name' in 'sneakers'